In [8]:

import geopandas as gpd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sketch
from shapely.geometry import Point
import time


# Configurar pandas para mostrar todas las filas y columnas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


data_frame = pd.read_table("df_ciber_modificado.csv", delimiter=',')




In [6]:
data_frame.sample(50)

,Fecha_hora,IP Origen,Destino IP,Puerto_origen,Puerto_destino,Protocolo,Longitud_paquete,Tipo_paquete,Tipo,Datos_carga_util,Indicadores_malware,Puntuaciones_anomalías,Alertas_advertencias,Tipo_ataque,Firma_ataque,respuesta_tomada,Nivel_severidad,Informacion_usuario,Informacion_dispositivo,Segmento_red,Datos_geolocalización,Información_proxy,Registros_cortafuegos,Alertas IDS (SisterC_intrusos/IPS (Sister_Preve_intrusos),Fuente_registro
1418,2020-11-06 15:20:02,107.74.134.37,116.22.18.222,34377,23079,TCP,1474,Control,FTP,Aceptó la opción de deudas para ser rechazada.Para hacer una vida física.Voliza el dolor de falla real con una ventaja de adulación.\nY los síntomas de todos.Y el placer del odio.,NaN,24.06,NaN,DDoS,Known Pattern B,Logged,High,Drishya Dara,"Mozilla/5.0 (Windows; U; Windows 95) AppleWebKit/534.44.5 (KHTML, like Gecko) Version/5.0.3 Safari/534.44.5",Segment C,"Rajkot, Gujarat",NaN,Log Data,NaN,Server
23201,2022-01-12 21:09:15,218.66.191.190,6.147.55.64,53651,59285,ICMP,1332,Control,HTTP,Dolores vitae corrupti cumque. Nam sequi eum minus atque aliquid est corporis.\nMaiores impedit assumenda doloribus. Veritatis ipsum incidunt tempore magnam.,IoC Detected,19.25,NaN,Intrusion,Known Pattern A,Logged,Low,Aaina Ahluwalia,Opera/8.32.(X11; Linux i686; tr-CY) Presto/2.9.170 Version/10.00,Segment B,"Burhanpur, West Bengal",217.188.231.36,Log Data,Alert Data,Server
28144,2021-04-09 13:12:00,121.41.223.189,92.188.80.129,11512,11496,ICMP,355,Data,DNS,Repellat pariatur illum voluptatum eos asperiores. Ipsa quasi sunt unde aliquam accusantium praesentium hic. Autem ducimus dolorum quod.,IoC Detected,28.27,Alert Triggered,Malware,Known Pattern A,Logged,Low,Divij Bose,Opera/8.59.(X11; Linux i686; sc-IT) Presto/2.9.160 Version/12.00,Segment A,"Panihati, Arunachal Pradesh",NaN,Log Data,Alert Data,Firewall
28522,2022-02-28 06:02:14,30.86.203.226,149.90.195.136,34565,56215,UDP,479,Data,DNS,Harum autem incidunt at optio voluptas laborum. Molestias facilis odio incidunt nam.,IoC Detected,69.79,NaN,Malware,Known Pattern B,Logged,Medium,Jivin Bora,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/531.1 (KHTML, like Gecko) Chrome/49.0.856.0 Safari/531.1",Segment C,"Madhyamgram, Maharashtra",NaN,NaN,Alert Data,Server
19628,2021-06-03 00:55:40,10.2.209.180,196.35.57.227,37534,6291,UDP,405,Control,HTTP,Totam unde totam enim. Facere nemo eveniet impedit aliquam tempore voluptas architecto.\nOptio at ullam blanditiis accusantium eveniet assumenda. Fuga et totam dicta. Magni inventore sunt delectus.,IoC Detected,86.99,Alert Triggered,DDoS,Known Pattern B,Blocked,Low,Manikya Bassi,"Mozilla/5.0 (Windows; U; Windows CE) AppleWebKit/531.16.1 (KHTML, like Gecko) Version/5.0 Safari/531.16.1",Segment C,"Tumkur, West Bengal",162.176.196.60,Log Data,Alert Data,Firewall
2751,2020-08-10 02:26:04,162.142.56.83,219.80.136.0,36689,28862,ICMP,676,Control,DNS,Que deben llevarlo a la mente excepto.Dijo eso pero que hacer.,NaN,2.39,NaN,Intrusion,Known Pattern B,Logged,Medium,Chirag Yadav,Mozilla/5.0 (X11; Linux i686; rv:1.9.7.20) Gecko/2976-09-18 04:50:30 Firefox/3.8,Segment C,"Panihati, Nagaland",90.99.39.105,NaN,NaN,Server
21397,2021-02-19 11:04:44,173.48.157.188,57.53.67.18,35668,55717,UDP,522,Control,FTP,Et facilis et nesciunt expedita fuga perspiciatis. Blanditiis expedita quo perspiciatis est dolorum est.\nVoluptatum qui eligendi fuga accusantium inventore repellat.,NaN,26.70,NaN,DDoS,Known Pattern B,Blocked,Low,Hrishita Bansal,"Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_10_4 rv:2.0; unm-US) AppleWebKit/534.5.7 (KHTML, like Gecko) Version/5.0.3 Safari/534.5.7",Segment C,"Cuttack, Gujarat",80.163.142.61,Log Data,Alert Data,Server
34135,2020-10-27 01:27:56,187.225.216.3,65.79.61.248,14142,56678,UDP,126,Control,FTP,Ratione aliquid recusandae doloribus ex. Non ex saepe dicta accusantium autem optio esse. Consequuntur quae possimus nam qui.,NaN,34.17,Alert Triggered,Intrusion,Known Pattern B,Logged,Low,Tiya Sinha,Opera/9.19.(X11; Linux x86_64; tt-R

In [9]:
def traducir_columna(df, columna, idioma_destino='es', chunk_size=100):
    translator = Translator()

    # Divide el DataFrame en partes más pequeñas (chunks)
    chunks = [df[i:i + chunk_size].copy() for i in range(0, len(df), chunk_size)]

    # Traduce cada parte y actualiza el DataFrame original
    for chunk in chunks:
        try:
            chunk[columna] = chunk[columna].apply(lambda x: translator.translate(x, dest=idioma_destino).text)
            df.loc[chunk.index, columna] = chunk[columna]
        except Exception as e:
            print(f"Error al traducir: {e}")

    return df

def traducir_columna(df, columna , idioma_destino='es', chunk_size=100, num_workers=8):
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        chunks = [df[i:i + chunk_size].copy() for i in range(0, len(df), chunk_size)]
        for chunk in chunks:
            chunk[columna] = list(executor.map(traducir_texto, chunk[columna]))
            df.loc[chunk.index, columna] = chunk[columna]

    return df

In [13]:
traducir_columna(data_frame,Datos_carga_util )
traducir_columna(data_frame,Datos_carga_util)

NameError: name 'Datos_carga_util' is not defined